# 0 Import

In [2]:
import os
import re
import csv
import napari
import tifffile
import numpy as np
import matplotlib.pyplot as plt

import bigfish
import bigfish.plot as plot
import bigfish.stack as stack
import bigfish.detection as detection
import bigfish.multistack as multistack

from copy import deepcopy
from dask.array.image import imread as imr
from bigfish.detection.utils import get_object_radius_pixel
from buildReferenceSpot import buildReferenceSpotFromImages
from runBigfishDetection import getSpotAndClusters, saveSpotsNPZ, reorderZstack, getSpotAndClusters_multi

In [3]:
from order_spot import * 

## Specify parameters 

In [4]:
voxelRadius = (700, 110, 110) # in nanometers
objectRadius = (700, 105, 105) # in nanometers

In [5]:
alpha = 0.5
beta = 2
gamma = 15

In [6]:
minFrame = int(input("Enter your max Frame: "))

Enter your max Frame: 1


In [7]:
maxFrame = int(input("Enter your max Frame: "))
maxFrame = maxFrame + 1 

Enter your max Frame: 120


## 1 Specify input

In [8]:
homeFolder = '/media/raphael/data/works/20231220_C3_14_2h_ofON/bleach_correction/expo' +'/'
nameKey = 'C3.14_ON_01_w1SPI-488_s'

imsQ = '4'

In [9]:
list = os.listdir(path=homeFolder + nameKey + imsQ +  '/' )

In [10]:
list.sort()
pattern = 'cell_'
cell_list = [string for string in list if pattern in string]

## 1.2 Use the '[]' to get rid of unnecessary folders 

In [11]:
cell_list[36:]

['C3.14_ON_01_w1SPI-488_s4_TS_cell_58_bleach_correction_sort_ref_spots0.5_2_15_thres_40.csv',
 'cell_10',
 'cell_12',
 'cell_13',
 'cell_18',
 'cell_20',
 'cell_21',
 'cell_22',
 'cell_23',
 'cell_24',
 'cell_25',
 'cell_26',
 'cell_28',
 'cell_29',
 'cell_31',
 'cell_32',
 'cell_33',
 'cell_34',
 'cell_36',
 'cell_37',
 'cell_40',
 'cell_41',
 'cell_42',
 'cell_43',
 'cell_45',
 'cell_46',
 'cell_49',
 'cell_50',
 'cell_51',
 'cell_52',
 'cell_53',
 'cell_54',
 'cell_55',
 'cell_57',
 'cell_58',
 'cell_59',
 'cell_6',
 'cell_60',
 'cell_62',
 'cell_64',
 'cell_8',
 'cell_9',
 'cell_exemple52',
 'cell_exemple_full52']

# Detection

In [12]:
def getDetectedPointsForFrame(pts_coordinates, frameNumer):
    sd = np.shape(pts_coordinates[frameNumer][:])
    pts_coords = np.empty([sd[0],sd[1]-1])
    for ii in range(np.shape(pts_coordinates[frameNumer][:])[0]):
        pts_coords[ii,:] = pts_coordinates[frameNumer][ii][1:]
    return pts_coords

def getDetectedClustersForFrame(pts_coordinates, frameNumer):
    sd = np.shape(pts_coordinates[frameNumer][:])
    pts_coords = np.empty([sd[0],sd[1]-3])
    for ii in range(np.shape(pts_coordinates[frameNumer][:])[0]):
        pts_coords[ii,:] = pts_coordinates[frameNumer][ii][1:3]
    return pts_coords

def set_pts_features(pts_layer, cls_layer, pts_coordinates, cluster_coordinate, step): #TxLayer
    # step is a 4D coordinate with the current slider position for each dim
    frameNumber = step[0]  # grab the leading ("time") coordinate
    pts_layer.data = getDetectedPointsForFrame(pts_coordinates,frameNumber)
    cls_layer.data = getDetectedClustersForFrame(cluster_coordinate,frameNumber)


In [14]:
for cell in cell_list[37:] : 

    cell_Crop_Folder = homeFolder+nameKey+imsQ+'/'+ cell+'/'
    pathToTimeSequenceCell = homeFolder+nameKey+imsQ+'/'+ cell+'/*.tif'
    sequenceCell = imr(pathToTimeSequenceCell)
    sequenceCell = sequenceCell
    mipSequenceCell = np.max(sequenceCell, axis=1)
    MaxTimePoint = sequenceCell.shape[0]
    
    
    images=[]    
    spots_list=[]

    spot_radius_px = detection.get_object_radius_pixel(
        voxel_size_nm=voxelRadius, 
        object_radius_nm=objectRadius, 
        ndim=3)
    cellnumber = cell[5:]
    print('cell : ',cellnumber)
    selectedThreshold = int(input("Enter your selected threshold : "))
    for t in range(minFrame,maxFrame,1):

        path = os.path.join(cell_Crop_Folder, nucleiStackForm+str(cell[5:])+'_t'+str(f"{t:03}")+".tif")
        rna = stack.read_image(path)
        images.append(rna)

    n=len(images)
    print("Total number of images : "+str(n))

    selectedThreshold_loop = selectedThreshold
    #get list of spot
    for rna in images:
        try : 
            # LoG filter
            rna_log = stack.log_filter(rna, sigma=spot_radius_px)

            # local maximum detection
            mask = detection.local_maximum_detection(rna_log, min_distance=spot_radius_px)

            # thresholding
            threshold = detection.automated_threshold_setting(rna_log, mask)

            selectedThreshold_loop = (selectedThreshold_loop) #*((100+var_list[i])/100) 
            #print(selectedThreshold_loop)
            spots_, _ = detection.spots_thresholding(rna_log, mask, float(selectedThreshold_loop))
            spots_list.append(spots_)
        except IndexError as e: 
            spots_, _ = detection.spots_thresholding(rna_log, mask, float(selectedThreshold))
            spots_list.append(spots_)
            print('Done')
            break 
    #sort list of spot 
    spots_list = sort_spot_list(spots_list,mipSequenceCell)
    #compute reference spot      
    reference_spot = buildReferenceSpotFromImages(images, spots_list, alpha=alpha, gamma=5, voxelSize=voxelRadius, objectSize=objectRadius)
    refSpot = deepcopy(reference_spot)
    
    #get spot and clusters 
    spotsFrame, clustersFrames, ThresholdFrames = getSpotAndClusters(images, 
                                                                 reference_spot, 
                                                                 cellnumber=cellnumber, 
                                                                 startTime=minFrame-1,
                                                                 stopTime=maxFrame-1,
                                                                 thresholdManual=selectedThreshold, 
                                                                 beta=beta, #2
                                                                 gamma=gamma, #15 
                                                                 numberOfSpots=2,
                                                                 radiusCluster=400, 
                                                                 voxelSize=voxelRadius, 
                                                                 objectSize=objectRadius,
                                                                 extensionMov='.tif')
    
    #compute clusters frames
    a = sort_clustersFrames(clustersFrames,mipSequenceCell,400,perc=1)
    b = merge_clusters_framev2(a,mipSequenceCell,diffxy=5,diffz=1)
    c = create_index2(b)   
    
    saveSpotsNPZ(np.array(spotsFrame, dtype=object), 
             np.array(clustersFrames, dtype=object), 
             np.array(ThresholdFrames, dtype=object), 
             cell[5:], 
             cell_Crop_Folder, 
             reference_spot,
             threshold = selectedThreshold
             ) 

    #save clusters frame in csv files 
    save_csv_file(homeFolder,nameKey,imsQ,cellNumber,selectedThreshold,alpha,beta,gamma,c)

cell :  10
Enter your selected threshold : 50
Total number of images : 120
image list found!
Found 5320
Found 5320 spots, max intensity = 142
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
done!
on 3: max() arg is an empty sequence                                            
on 23: max() arg is an empty sequence                                           
on 25: max() arg is an empty sequence                                           
on 27: max() arg is an empty sequence                                           
on 28: max() arg is an empty sequence                                           
on 31: max() arg is an empty sequence                                        

/home/raphael/miniconda3/envs/bigfishlive/lib/python3.9/site-packages/bigfish/detection/spot_detection.py:586: RuntimeWarning: divide by zero encountered in log
  count_spots = np.log([np.count_nonzero(value_spots > t)


image list found!
Found 5182
Found 5182 spots, max intensity = 148
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


/home/raphael/miniconda3/envs/bigfishlive/lib/python3.9/site-packages/bigfish/detection/spot_detection.py:586: RuntimeWarning: divide by zero encountered in log
  count_spots = np.log([np.count_nonzero(value_spots > t)


33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
done!
on 0: max() arg is an empty sequence                                            
on 1: max() arg is an empty sequence                                            
on 2: max() arg is an empty sequence                                            
on 3: max() arg is an empty sequence                                            
on 4: max() arg is an empty sequence                                            
on 5: max() arg is an empty sequence                                            
on 6: max() arg is an empty sequence                                            
on 7: max() arg is an empty sequence                                            
on 8: max() arg is an empty sequence                             

...

/home/raphael/miniconda3/envs/bigfishlive/lib/python3.9/site-packages/bigfish/detection/spot_detection.py:586: RuntimeWarning: divide by zero encountered in log
  count_spots = np.log([np.count_nonzero(value_spots > t)


118


/home/raphael/miniconda3/envs/bigfishlive/lib/python3.9/site-packages/bigfish/detection/spot_detection.py:586: RuntimeWarning: divide by zero encountered in log
  count_spots = np.log([np.count_nonzero(value_spots > t)


119
done!
sort cluster frame |████████████████████████████████████████| 120/120 [100%] in 
cell :  18
Enter your selected threshold : 105
Total number of images : 120
image list found!
Found 4020
Found 4020 spots, max intensity = 307
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
done!
on 0: max() arg is an empty sequence                                            
on 37: max() arg is an empty sequence                                           
on 39: max() arg is an empty sequence                                           
on 40: max() arg is an empty sequence                                           
on 45: max() arg is an empty sequence                             

KeyboardInterrupt: Interrupted by user